In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tool Wear Detection: Batch Prediction

## Setup

### Project ID

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: mfg-solutions-ce-demo-2


## Explore MDE BigQuery

https://cloud.google.com/bigquery/docs/visualize-jupyter#query_notebook_data_using_the_bigquery_magic_command

https://googleapis.dev/python/bigquery/latest/magics.html

In [19]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
    schema_name
FROM
    `{PROJECT_ID}.INFORMATION_SCHEMA.SCHEMATA`;
'''

results = bq_client.query(query)

results.to_dataframe()

,schema_name
0,ml
1,sfp_data
2,mde_looker_pdt


In [21]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
    *
FROM
    `{PROJECT_ID}.sfp_data.INFORMATION_SCHEMA.TABLES`;
'''

results = bq_client.query(query)

results.to_dataframe()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,mfg-solutions-ce-demo-2,sfp_data,NumericDataSeries,BASE TABLE,YES,NO,2022-08-22 20:24:01.338000+00:00,None,None,None,NaT,CREATE TABLE `mfg-solutions-ce-demo-2.sfp_data...,NULL,NaT
1,mfg-solutions-ce-demo-2,sfp_data,InsertErrors,BASE TABLE,YES,NO,2022-08-22 20:24:00.950000+00:00,None,None,None,NaT,CREATE TABLE `mfg-solutions-ce-demo-2.sfp_data...,NULL,NaT
2,mfg-solutions-ce-demo-2,sfp_data,ComponentDataSeries,BASE TABLE,YES,NO,2022-08-22 20:24:01.333000+00:00,None,None,None,NaT,CREATE TABLE `mfg-solutions-ce-demo-2.sfp_data...,NULL,NaT
3,mfg-solutions-ce-demo-2,sfp_data,DiscreteDataSeries,BASE TABLE,YES,NO,2022-08-22 20:24:01.130000+00:00,None,None,None,NaT,CREATE TABLE `mfg-solutions-ce-demo-2.sfp_data...,NULL,NaT
4,mfg-solutions-ce-demo-2,sfp_data,ContinuousDataSeries,BASE TABLE,YES,NO,2022-08-22 20:24:00.950000+00:00,None,None,None,NaT,CREATE TABLE `mfg-solutions-ce-demo-2.sfp_data...,NULL,NaT
5,mfg-solutions-ce-demo-2,sfp_data,OperationsDashboard,BASE TABLE,YES,NO,2022-08-22 20:24:01.192000+00:00,None,None,None,NaT,CREATE TABLE `mfg-solutions-ce-demo-2.sfp_data...,NULL,NaT


In [23]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
    *
FROM
    `{PROJECT_ID}.sfp_data.NumericDataSeries`
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
LIMIT 3;
'''

results = bq_client.query(query)

results.to_dataframe()

,tagName,edgeTagName,eventTimestamp,value,payload,payloadQualifier,metadata,ingestTimestamp,payloadKV,metadataKV,payloadQualifierKV,messageId,id
0,cncmilling_y1_positiondiff,cncmilling_y1_positiondiff,2022-10-03 19:33:40.871000+00:00,0E-38,"{""value"":0}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/4/local/organization/1"":{""companyId""...",2022-10-03 19:33:45.104665+00:00,"[{'key': 'value', 'value': '0'}]",[{'schemaIdentifier': 'cncmill/4/local/organiz...,"[{'key': 'headers.payload', 'value': 'numeric-...",5817490215985751,f07e7e12-325f-4b3e-bacd-eda999965c63
1,cncmilling_y1_velocitydiff,cncmilling_y1_velocitydiff,2022-10-03 19:33:40.871000+00:00,0E-38,"{""value"":0}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/4/local/organization/1"":{""companyId""...",2022-10-03 19:33:45.104800+00:00,"[{'key': 'value', 'value': '0'}]",[{'schemaIdentifier': 'cncmill/4/local/organiz...,"[{'key': 'headers.payload', 'value': 'numeric-...",5817490215985750,865fc530-5dd5-40ed-9efe-84a2d36fd486
2,cncmilling_y1_currentfeedback,cncmilling_y1_currentfeedback,2022-10-03 19:38:30.901000+00:00,-1.04000000000000000000000000000000000000,"{""value"":-1.04}","{""headers"":{""payload"":""numeric-payload-default...","{""cncmill/4/local/organization/1"":{""companyId""...",2022-10-03 19:38:33.343461+00:00,"[{'key': 'value', 'value': '-1.04'}]",[{'schemaIdentifier': 'cncmill/4/local/organiz...,"[{'key': 'headers.payload', 'value': 'numeric-...",5817534958507131,1936f60a-fe13-4102-9662-b0c5210ad3d4


In [24]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
    *
FROM
    `{PROJECT_ID}.sfp_data.DiscreteDataSeries`
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
LIMIT 3;
'''

results = bq_client.query(query)

results.to_dataframe()

,tagName,edgeTagName,eventTimestamp,eventType,payload,payloadQualifier,metadata,ingestTimestamp,payloadKV,metadataKV,payloadQualifierKV,messageId,id
0,cncmilling_tool_condition,cncmilling_tool_condition,2022-10-03 19:40:30.912000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/4/local/organization/1"":{""companyId""...",2022-10-03 19:40:33.740179+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/4/local/organiz...,"[{'key': 'headers.payload', 'value': 'complex-...",5817572093459648,14f0ae05-dfa5-4bb4-86df-dde2dd852fe8
1,cncmilling_tool_condition,cncmilling_tool_condition,2022-10-03 19:40:35.911000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/4/local/organization/1"":{""companyId""...",2022-10-03 19:40:38.141457+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/4/local/organiz...,"[{'key': 'headers.payload', 'value': 'complex-...",5817541108147825,93f6e90c-93e3-41b6-8996-82df09c0d161
2,cncmilling_tool_condition,cncmilling_tool_condition,2022-10-03 19:40:40.911000+00:00,None,"""unworn""","{""headers"":{""payload"":""complex-payload-default...","{""cncmill/4/local/organization/1"":{""companyId""...",2022-10-03 19:40:43.885214+00:00,"[{'key': 'value', 'value': 'unworn'}]",[{'schemaIdentifier': 'cncmill/4/local/organiz...,"[{'key': 'headers.payload', 'value': 'complex-...",5817511631563521,273052ad-eddb-4bdd-a563-63ec071d4c49


In [25]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
  DISTINCT tagName 
FROM 
  `{PROJECT_ID}.sfp_data.NumericDataSeries`
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
  AND STARTS_WITH(tagName, 'cncmilling_')
ORDER BY tagName;
'''

results = bq_client.query(query)

results.to_dataframe()

,tagName
0,cncmilling_clamp_pressure
1,cncmilling_feedrate
2,cncmilling_m1_current_feedrate
3,cncmilling_m1_current_program_number
4,cncmilling_m1_sequence_number
...,...
58,cncmilling_z1_dcbusvoltage
59,cncmilling_z1_outputcurrent
60,cncmilling_z1_outputvoltage
61,cncmilling_z1_positiondiff


In [26]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
  DISTINCT tagName 
FROM 
  `{PROJECT_ID}.sfp_data.DiscreteDataSeries`
WHERE
  DATE(eventTimestamp) = CURRENT_DATE()
  AND STARTS_WITH(tagName, 'cncmilling_')
ORDER BY tagName;
'''

results = bq_client.query(query)

results.to_dataframe()

,tagName
0,cncmilling_material
1,cncmilling_tool_condition


In [27]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.ml`
  OPTIONS (
    description = 'Dataset for storing machine learning artifacts',
    location = 'us'
  );
'''

results = bq_client.query(query)

results.to_dataframe()

""


In [28]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
CREATE OR REPLACE TABLE `{PROJECT_ID}.ml.tool_wear_tags` (
  tagName STRING,
  featureName STRING
) OPTIONS (
    description = 'Mapping between tags and feature for tool wear classification'
);
'''

results = bq_client.query(query)

results.to_dataframe()

""


Initialize tags-to-feature mapping table.

In [29]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
INSERT `{PROJECT_ID}.ml.tool_wear_tags` (tagName, featureName)
VALUES 
('cncmilling_clamp_pressure', 'clamp_pressure')
, ('cncmilling_feedrate', 'feedrate')
, ('cncmilling_m1_current_feedrate', 'M1_CURRENT_FEEDRATE')
, ('cncmilling_m1_current_program_number', 'M1_CURRENT_PROGRAM_NUMBER')
, ('cncmilling_m1_sequence_number', 'M1_sequence_number')
, ('cncmilling_machining_process', 'Machining_Process')
, ('cncmilling_s1_accelerationdiff', 'S1_AccelerationDiff')
, ('cncmilling_s1_actualacceleration', 'S1_ActualAcceleration')
, ('cncmilling_s1_actualposition', 'S1_ActualPosition')
, ('cncmilling_s1_actualvelocity', 'S1_ActualVelocity')
, ('cncmilling_s1_commandacceleration', 'S1_CommandAcceleration')
, ('cncmilling_s1_commandposition', 'S1_CommandPosition')
, ('cncmilling_s1_commandvelocity', 'S1_CommandVelocity')
, ('cncmilling_s1_currentfeedback', 'S1_CurrentFeedback')
, ('cncmilling_s1_dcbusvoltage', 'S1_DCBusVoltage')
, ('cncmilling_s1_outputcurrent', 'S1_OutputCurrent')
, ('cncmilling_s1_outputpower', 'S1_OutputPower')
, ('cncmilling_s1_outputvoltage', 'S1_OutputVoltage')
, ('cncmilling_s1_positiondiff', 'S1_PositionDiff')
, ('cncmilling_s1_systeminertia', 'S1_SystemInertia')
, ('cncmilling_s1_velocitydiff', 'S1_VelocityDiff')
, ('cncmilling_x1_accelerationdiff', 'X1_AccelerationDiff')
, ('cncmilling_x1_actualacceleration', 'X1_ActualAcceleration')
, ('cncmilling_x1_actualposition', 'X1_ActualPosition')
, ('cncmilling_x1_actualvelocity', 'X1_ActualVelocity')
, ('cncmilling_x1_commandacceleration', 'X1_CommandAcceleration')
, ('cncmilling_x1_commandposition', 'X1_CommandPosition')
, ('cncmilling_x1_commandvelocity', 'X1_CommandVelocity')
, ('cncmilling_x1_currentfeedback', 'X1_CurrentFeedback')
, ('cncmilling_x1_dcbusvoltage', 'X1_DCBusVoltage')
, ('cncmilling_x1_outputcurrent', 'X1_OutputCurrent')
, ('cncmilling_x1_outputpower', 'X1_OutputPower')
, ('cncmilling_x1_outputvoltage', 'X1_OutputVoltage')
, ('cncmilling_x1_positiondiff', 'X1_PositionDiff')
, ('cncmilling_x1_velocitydiff', 'X1_VelocityDiff')
, ('cncmilling_y1_accelerationdiff', 'Y1_AccelerationDiff')
, ('cncmilling_y1_actualacceleration', 'Y1_ActualAcceleration')
, ('cncmilling_y1_actualposition', 'Y1_ActualPosition')
, ('cncmilling_y1_actualvelocity', 'Y1_ActualVelocity')
, ('cncmilling_y1_commandacceleration', 'Y1_CommandAcceleration')
, ('cncmilling_y1_commandposition', 'Y1_CommandPosition')
, ('cncmilling_y1_commandvelocity', 'Y1_CommandVelocity')
, ('cncmilling_y1_currentfeedback', 'Y1_CurrentFeedback')
, ('cncmilling_y1_dcbusvoltage', 'Y1_DCBusVoltage')
, ('cncmilling_y1_outputcurrent', 'Y1_OutputCurrent')
, ('cncmilling_y1_outputpower', 'Y1_OutputPower')
, ('cncmilling_y1_outputvoltage', 'Y1_OutputVoltage')
, ('cncmilling_y1_positiondiff', 'Y1_PositionDiff')
, ('cncmilling_y1_velocitydiff', 'Y1_VelocityDiff')
, ('cncmilling_z1_accelerationdiff', 'Z1_AccelerationDiff')
, ('cncmilling_z1_actualacceleration', 'Z1_ActualAcceleration')
, ('cncmilling_z1_actualposition', 'Z1_ActualPosition')
, ('cncmilling_z1_actualvelocity', 'Z1_ActualVelocity')
, ('cncmilling_z1_commandacceleration', 'Z1_CommandAcceleration')
, ('cncmilling_z1_commandposition', 'Z1_CommandPosition')
, ('cncmilling_z1_commandvelocity', 'Z1_CommandVelocity')
, ('cncmilling_z1_currentfeedback', 'Z1_CurrentFeedback')
, ('cncmilling_z1_dcbusvoltage', 'Z1_DCBusVoltage')
, ('cncmilling_z1_outputcurrent', 'Z1_OutputCurrent')
, ('cncmilling_z1_outputvoltage', 'Z1_OutputVoltage')
, ('cncmilling_z1_positiondiff', 'Z1_PositionDiff')
, ('cncmilling_z1_velocitydiff', 'Z1_VelocityDiff')
, ('cncmilling_tool_condition', 'tool_condition')
, ('cncmilling_material', 'material');
'''

results = bq_client.query(query)

results.to_dataframe()

""


In [31]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
  *
FROM
  `{PROJECT_ID}.ml.tool_wear_tags`
ORDER BY
  tagName 
LIMIT 5;
'''

results = bq_client.query(query)

results.to_dataframe()

,tagName,featureName
0,cncmilling_clamp_pressure,clamp_pressure
1,cncmilling_feedrate,feedrate
2,cncmilling_m1_current_feedrate,M1_CURRENT_FEEDRATE
3,cncmilling_m1_current_program_number,M1_CURRENT_PROGRAM_NUMBER
4,cncmilling_m1_sequence_number,M1_sequence_number


In [33]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
CREATE OR REPLACE PROCEDURE `{PROJECT_ID}.ml.create_prediction_features_view`(view_name STRING)
BEGIN
    DECLARE select_expr STRING;
    DECLARE pivot_expr STRING;
    DECLARE query STRING;
    DECLARE last_prediction_timestamp DEFAULT (
      SELECT
        IFNULL(
          MAX(eventTimestamp),
          TIMESTAMP_SECONDS(0)
        ) AS last_prediction_timestamp
      FROM
        `sfp_data.DiscreteDataSeries`
      WHERE
        DATE(eventTimestamp) >= DATE_FROM_UNIX_DATE(0)
        AND tagName = 'tool_wear_predictions'
    );
    
    -- Dynamically create select expressions
    SET select_expr = (
      WITH SelectExpr AS (
        SELECT 
          CASE
            WHEN EXISTS (SELECT DISTINCT tagName FROM `sfp_data.DiscreteDataSeries` d WHERE d.eventTimestamp > last_prediction_timestamp AND d.tagName = t.tagName)
              THEN FORMAT(', JSON_EXTRACT_SCALAR(payload_%s, "$") AS %s', t.tagName, t.featureName)
            WHEN EXISTS (SELECT DISTINCT tagName FROM `sfp_data.NumericDataSeries` n WHERE n.eventTimestamp > last_prediction_timestamp AND n.tagName = t.tagName)
              THEN FORMAT(', JSON_EXTRACT_SCALAR(payload_%s, "$.value") AS %s', t.tagName, t.featureName)
            ELSE ''
          END AS expr
        FROM
          `ml.tool_wear_tags` t
      )
      SELECT 
        STRING_AGG(SelectExpr.expr, ' ')
      FROM 
        SelectExpr
    );
    
    -- Dynamically create pivot expression
    SET pivot_expr = (
      SELECT
        STRING_AGG(DISTINCT CONCAT("'", tagName, "'"))
      FROM
        `ml.tool_wear_tags`
    );
    
    -- Format query using dynamic select and pivot expressions
    SET query = (
      SELECT FORMAT(r"""
        CREATE OR REPLACE VIEW %s AS
        WITH metadata AS (
            SELECT
              d.eventTimestamp
              , CASE
                  WHEN (REGEXP_CONTAINS(mkv.schemaIdentifier, r'^(cncmill/\d+/local/cncmill/\d+)$') AND mkv.key = 'material')
                    THEN 'cncmilling_material'
                  ELSE NULL
                END AS tagName
              , TO_JSON_STRING(mkv.value) AS payload
            FROM
              `sfp_data.DiscreteDataSeries` d, d.metadataKV mkv
            WHERE
              TIMESTAMP_TRUNC(d.eventTimestamp, SECOND) > "%t"
              AND d.tagName = 'cncmilling_tool_condition'
              AND ARRAY_LENGTH(metadataKV) != 0
        )
        SELECT 
          eventTimestamp
          %s
          , JSON_EXTRACT_SCALAR(payload_cncmilling_material, "$") AS material
        FROM 
        (
          SELECT
            TIMESTAMP_TRUNC(n.eventTimestamp, SECOND) AS eventTimestamp
            , n.tagName
            , n.payload
          FROM
            `sfp_data.NumericDataSeries` n
          INNER JOIN
            `ml.tool_wear_tags` ntag
          ON
            n.tagName = ntag.tagName
          WHERE
            TIMESTAMP_TRUNC(n.eventTimestamp, SECOND) > "%t"
          
          UNION ALL
          
          SELECT
            TIMESTAMP_TRUNC(d.eventTimestamp, SECOND) AS eventTimestamp
            , d.tagName
            , d.payload
          FROM
            `sfp_data.DiscreteDataSeries` d
          INNER JOIN
            `ml.tool_wear_tags` dtag
          ON
            d.tagName = dtag.tagName
          WHERE
            TIMESTAMP_TRUNC(d.eventTimestamp, SECOND) > "%t"
          
          UNION ALL
          
          SELECT
            TIMESTAMP_TRUNC(metadata.eventTimestamp, SECOND) AS eventTimestamp
            , metadata.tagName
            , metadata.payload
          FROM
            metadata
          INNER JOIN
            `ml.tool_wear_tags` mtag
          ON
            metadata.tagName = mtag.tagName
        )
        PIVOT
        (
          ANY_VALUE(payload) as payload
          FOR tagName
          IN (
            %s
          )
        )
    """,
        view_name,
        last_prediction_timestamp, 
        select_expr, 
        last_prediction_timestamp, 
        last_prediction_timestamp, 
        pivot_expr)
    );

    -- Execute query
    EXECUTE IMMEDIATE query;
END
'''

results = bq_client.query(query)

results.to_dataframe()

""


In [15]:
from datetime import datetime

current_time_string = datetime.now().strftime('%Y%m%d_%H%M%S')
view_name = f'ml.prediction_features_{current_time_string}'
bq_params = {'view_name': view_name}

view_name

'ml.prediction_features_20221003_190700'

In [17]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''CALL `{PROJECT_ID}.ml.create_prediction_features_view`('{view_name}');'''

results = bq_client.query(query)

results.to_dataframe()

BadRequest: 400 Query error: CREATE VIEW has columns with duplicate name material at [mfg-solutions-ce-demo-2.ml.create_prediction_features_view:3:9]

Location: US
Job ID: 062b52ac-abec-41f3-8262-7fd5499e797a


In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
  * 
FROM 
  `{PROJECT_ID}.ml.INFORMATION_SCHEMA.VIEWS`
WHERE
  table_name = '{view_name.split('.')[1]}';
'''

results = bq_client.query(query)
df = results.to_dataframe()

print(df['view_definition'].values[0])

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT *
FROM `{PROJECT_ID}.{view_name}`
ORDER BY eventTimestamp DESC
LIMIT 10;
'''

results = bq_client.query(query)

results.to_dataframe()

## Vertex AI Batch Prediction

In [ ]:
from datetime import datetime

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import Model
from google.cloud.aiplatform_v1beta1.services.job_service import JobServiceClient
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

vertex_ai.init(project=PROJECT_ID)

### Global variables

In [ ]:
LOCATION = 'us-central1'
API_ENDPOINT = f'{LOCATION}-aiplatform.googleapis.com'
CLIENT_OPTIONS = {"api_endpoint": API_ENDPOINT}
PARENT = f"projects/{PROJECT_ID}/locations/{LOCATION}"

BQ_ML_DATASET = 'ml'
BQ_PREDICTION_INPUT_TABLE = f'{BQ_ML_DATASET}.unpredicted_data'
NAME_PREFIX = 'tool_wear'
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
MODEL_NAME = "[your-model-name]" # Format: 'projects/{project}/locations/{location}/models/{model_id}'

if MODEL_NAME == "" or MODEL_NAME is None or MODEL_NAME == "[your-model-name]":
    # Get latest updated model with prefix
    models = Model.list(
        order_by='updateTime desc',
    )

    MODEL_NAME = None
    for m in models:
        if m.display_name.startswith(NAME_PREFIX):
            MODEL_NAME = m.resource_name
            print(f'Vertex AI model found: {m.display_name}')
            break

    if MODEL_NAME is None:
        print(f'Vertex AI model with prefix "{NAME_PREFIX}" not found.'
              f'Please search using a new prefix or locate the model resource name manually.')

In [ ]:
TRAINING_DATASET_NAME = "[your-dataset-name]" # Format: 'projects/{project}/locations/{location}/datasets/{dataset_id}'

if TRAINING_DATASET_NAME == "" or TRAINING_DATASET_NAME is None or TRAINING_DATASET_NAME == "[your-dataset-name]":
    # Get latest updated dataset with prefix
    client = vertex_ai.gapic.DatasetServiceClient(client_options=CLIENT_OPTIONS)
    datasets = client.list_datasets(request={
        'parent': PARENT,
        'order_by': 'update_time desc',
    })
    
    TRAINING_DATASET_NAME = None
    for d in datasets:
        if d.display_name.startswith(NAME_PREFIX):
            TRAINING_DATASET_NAME = d.name
            print(f'Vertex AI dataset found: {d.display_name}')
            break
    
    if TRAINING_DATASET_NAME is None:
        print(f'Vertex AI dataset with prefix "{NAME_PREFIX}" not found.'
              f'Please search using a new prefix or locate the dataset resource name manually.')

### Create Batch Prediction Job

Reference: https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#aiplatform_create_batch_prediction_job_bigquery_sample-python

In [ ]:
DISPLAY_NAME = f'tool_wear_{TIMESTAMP}'
model_parameters_dict = {}
MODEL_PARAMETERS = json_format.ParseDict(model_parameters_dict, Value())
INSTANCE_FORMAT = 'bigquery' # https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.models#Model.FIELDS.supported_input_storage_formats
BQ_SOURCE_URI = f'bq://{PROJECT_ID}.{view_name}' # Format: bq://projectId.bqDatasetId.bqTableId
PREDICTIONS_FORMAT = 'jsonl' # https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.models#Model.FIELDS.supported_output_storage_formats
GCS_OUTPUT_URI_PREFIX = f'gs://{PROJECT_ID}-batch-ingestion/tool_wear_predictions/' # Format: gs://bucket_name/folder_name
MACHINE_TYPE = 'n1-standard-2'
MIN_NODES = 1
MAX_NODES = 3

# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = JobServiceClient(client_options=CLIENT_OPTIONS)

batch_prediction_job = {
    "display_name": DISPLAY_NAME,
    "model": MODEL_NAME,
    "model_parameters": MODEL_PARAMETERS,
    "input_config": {
        "instances_format": INSTANCE_FORMAT,
        "bigquery_source": {"input_uri": BQ_SOURCE_URI},
    },
    "output_config": {
        "predictions_format": PREDICTIONS_FORMAT,
        "gcs_destination": {"output_uri_prefix": GCS_OUTPUT_URI_PREFIX},
    },
    "dedicated_resources": {
        'machine_spec': {
            "machine_type": MACHINE_TYPE,
        },
        'starting_replica_count': MIN_NODES,
        'max_replica_count': MAX_NODES,
    },
    "generate_explanation": True,
    "explanation_spec": {
        "parameters": {
            "sampled_shapley_attribution": {
                "path_count": 3
            }
        }
    },
    # 'model_monitoring_config': {
    #     'alert_config': {
    #         'enable_logging': True
    #     },
    #     'objective_configs': [
    #         {
    #             'training_dataset': {
    #                 'dataset': TRAINING_DATASET_NAME
    #             },
    #         }
    #     ],
    # }
}


response = client.create_batch_prediction_job(
    parent=PARENT, batch_prediction_job=batch_prediction_job
)

print("response:", response.name)

## Explore Predictions Data

You may have to wait for the batch prediction job to finish. 

In [34]:
from google.cloud import bigquery

bq_client = bigquery.Client()

query = f'''
SELECT
  *
FROM
  `{PROJECT_ID}.sfp_data.DiscreteDataSeries`
WHERE
  DATE(eventTimestamp) > DATE_SUB(CURRENT_DATE(),INTERVAL 1 DAY)
  AND tagName = 'tool_wear_predictions'
ORDER BY
  eventTimestamp DESC
LIMIT 5;
'''

results = bq_client.query(query)

results.to_dataframe()

,tagName,edgeTagName,eventTimestamp,eventType,payload,payloadQualifier,metadata,ingestTimestamp,payloadKV,metadataKV,payloadQualifierKV,messageId,id


## Clean Up

In [ ]:
# from google.cloud import bigquery

# bq_client = bigquery.Client()
# query = f'''DROP VIEW IF EXISTS {view_name};'''
# results = bq_client.query(query)

# results.to_dataframe()